# 570 Interim Deliverable 2 

## MIP formulation

#### Input data:  
$$\begin{aligned}
\text{Section information:}
&& I & = \text{{all course sections}} \\
&& student & = \text{{predicted number of students who will register for each section}} \\
&& PS & = \text{{number of sessions for each section}}  \\
&& TS & = \text{{the timelength of each session in hours (e.g., 1.5, 2, 3)}}  \\
&& I_2 & = \text{{sections with 2 sessions a week (i.e., MW and TH)}} \\
\\
\text{Classroom information:}
&& J & = \text{{classrooms}} \\
&& seat & = \text{{number of seats in each classroom}}  \\
\\
\text{Day-time information:}
&& P & = \text{{day of week (i.e., M T W H F)}}  \\
&& T & = \text{{start time of a section}} \\
\end{aligned}$$

#### Decision variable:
Let the binary decision variable $X_{ijpt}$ denotes whether we assign course section $i$ to classroom $j$ at day $p$ starting at time $t$.

#### Objective and constraints:
Our goal is to maximize the capacity utilization rate. The linear program is thus:
$$\begin{aligned}
\text{maximize} && \sum_{i\in I}\sum_{j\in J}\sum_{p\in P}\sum_{t\in T}{X_{ijpt}\frac{student_i}{seat_j}} \\
\text{subject to:} \\
\
\text{No start time conflict for classes:} && X_{ijpt} + X_{(I-i)jpt} & \le 1 && \forall i \in I, j \in J, p \in P, t \in T\\
\\
\text{No class conflict when a class is in session:} && X_{ijpt} + X_{i'jp(t+c)}& \le 1 && \forall i,i' \in I; j \in J; p \in P; t \in T; c \in \{1,2,...,TS_i*2-1\}\\
\\
\text{All required sessions must be scheduled:} && \sum_{j\in J}\sum_{p\in P}\sum_{t\in T}{X_{ijpt}} & = PS_i && \forall i \in I\\
\\
\text{Number of students must not exceed total seats:} && X_{ijpt} * student_i & \le seat_j && \forall i \in I, j \in J, p \in P, t \in T \\
\\
\text{No class scheduled  to last beyond school time (i.e., 9.30PM):} && X_{ijpt'} & = 0 && \forall i \in I, j \in J, p \in P, t' \in \{T_{max}-(TS_i*2)+2,...,T_{max}\} \\
\\
\text{Binding clause for courses with 2 sessions (MW):} && X_{ij'M't} & = X_{ij'W't} && \forall i \in I_2, j \in J, t \in T\\
\text{Binding clause for courses with 2 sessions (TH):} && X_{ij'T't} & = X_{ij'H't} && \forall i \in I_2, j \in J, t \in T\\
\\
\text{No Friday class for courses with 2 sessions:} && X_{ij'F't} & = 0 && \forall i \in I_2, j \in J, t \in T \\
\end{aligned}$$

# Python Code

In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
import datetime
from datetime import time
from datetime import timedelta
import math

# for the room capacity file i remove the first record because it is same as the 7th record and will make some error in next step
capacity=pd.read_excel("Marshall_Room_Capacity_Chart.xlsx")
course_enrollment=pd.read_excel("Marshall_Course_Enrollment_1516_1617.xlsx") # for student prediction
#course=pd.read_excel("Marshall_Course_Enrollment_1516_1617_small.xlsx")
#section_info=pd.read_excel("section information.xlsx")

In [35]:
course_enrollment.head()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Begin Time,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title
0,ACCT-370,ACCT,370,ACCT,10:00:00,F,11:50:00,"Hopkins, Merle, W",3.783354e+09,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14029,1,20153,External Financial Reporting Issues
1,ACCT-370,ACCT,370,ACCT,08:00:00,MW,09:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14025,1,20153,External Financial Reporting Issues
2,ACCT-370,ACCT,370,ACCT,10:00:00,MW,11:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14026,1,20153,External Financial Reporting Issues
3,ACCT-370,ACCT,370,ACCT,12:00:00,MW,13:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14027,1,20153,External Financial Reporting Issues
4,ACCT-371,ACCT,371,ACCT,10:00:00,F,11:50:00,NaN,NaN,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14044,1,20153,Introduction to Accounting Systems


## Extra data from original dataset

In [2]:
# choose 20153 term as sample
data=course_enrollment.loc[course_enrollment.loc[:,"Term"]==20153,("Course","Section","First Days","First Begin Time","First End Time","First Room","First Instructor")]
data.columns=["Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstRoom","FirstInstructor"]
# we only use those M/T/W/H/F/MW/TH
data=data[(data.FirstDays=="M")|(data.FirstDays=="T")|(data.FirstDays=="W")|(data.FirstDays=="H")|(data.FirstDays=="F")|(data.FirstDays=="MW")|(data.FirstDays=="TH")]
data=data[(data.FirstRoom!="ONLINE")&(data.FirstRoom!="DEN@Viterbi")]
# remove the record that have nan in first begintime
data=data[(data.FirstBeginTime==data.FirstBeginTime)|(data.FirstEndTime==data.FirstEndTime)]
# remove the record that have nan in professor
data=data[data.FirstInstructor==data.FirstInstructor]
data=data.loc[:,("Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstInstructor")]

In [3]:
# timelength
timelength=[]
for i in data.index:
    #print i
    #print data.loc[i,:]
    #print data.loc[i,:]
    time1=data.loc[i,"FirstEndTime"]
    #print type(time1)
    time2=data.loc[i,"FirstBeginTime"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)

In [4]:
# pattern
pattern=[]
for i in data.index:
    day=data.loc[i,"FirstDays"]
    if (day=="M") or (day=="T") or (day=="W") or (day=="H") or (day=="F"):
        #print day
        pattern.append(1)
    elif (day=="MW") or (day=="TH"):
        #print day
        pattern.append(2)

Use the "section_info=section_info[section_info.timelength<=2]" below to control the size of dataset (change value of 2), i only tried small one

In [5]:
# build section_info table
section_info=pd.DataFrame({"course":data["Course"],"section":data["Section"],"pattern":pattern,"timelength":timelength,"FirstInstructor":data["FirstInstructor"]})
# we only use those timelength less than 1.5 hours
section_info=section_info[section_info.timelength<=3]
# build table of professor and sections
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
print max(section_info["timelength"])
print len(section_info)
#print section_info.index
# for change the index, do not delete 
section_info.index=range(len(section_info))
# get how many sessions we have in this sechedule
numofsections=sum(section_info["pattern"])
numofprof=len(Prof)

3
135


### change the size of sample dataset

In [34]:
# select a subset of section id 
section_info=section_info.loc[range(5),:]
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
numofsections=sum(section_info["pattern"])
numofprof=len(Prof)
print numofsections
print numofprof

8
3


## Build variables

In [1]:
# What we need to extract from data
# prediction about student registeration for every section
# how many section we need to schedule
# the pattern, time length of each section
# classroom capacity

# Variables
# course name
C={}
index=0
for i in section_info["section"]:
    C[i]=section_info.loc[section_info.loc[:,"section"]==i,"course"].get(index)
    index+=1
# section of course
I=section_info["section"]
# classroom
J=capacity["Room"]
# pattern of session (less than catagories in origional dataset)
P=["Monday","Tuesday","Wednesday","Thursday","Friday"]
P=pd.Series(i for i in P)
# start time of session (from 8:00am to 9:00pm,so the real classtime is between 8:00am to 10:00pm)
T1={}
time = datetime.timedelta(hours=8,minutes=0, seconds=0)
for i in range(27):
    T1[str(time+timedelta(hours=0.5*i))]=i
T=pd.Series(i for i in range(27))

# students' registeration prediction
student={}
for i in I:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)
#student=pd.Series(i for i in student)

# seat
seat={}
for index,row in capacity.iterrows():
    seat[row["Room"]]=row["Size"]
#seat=pd.Series(i for i in seat)
    
# pattern of each section
PS={}
index=0
for i in I:
    PS[i]=section_info.loc[section_info.loc[:,"section"]==i,"pattern"].get(index)
    index+=1
#PS=pd.Series(i for i in PS)
    
# timelength of each section
TS={}
index=0
for i in I:
    TS[i]=section_info.loc[section_info.loc[:,"section"]==i,"timelength"].get(index)
    index+=1
#TS=pd.Series(i for i in TS)

#define primetime
primetime=range(4,21)

NameError: name 'section_info' is not defined

## Build Model

In [37]:
# Building model
mod=grb.Model()

## Decision Variables

In [ ]:
X={}
for i in I: # section id
    for j in J: # classroom
        for p in P: # day of week
             for t in T: # start timeslot of sesstion
                    X[i,j,p,t]=mod.addVar(vtype=grb.GRB.BINARY, name='x[{0},{1},{2},{3}]'.format(i,j,p,t))

                    
# add one more variable PD[f,p,b], f is faculty(professor), p is day of week, b is the building
# PD[f,p,b]=1 means that prof f in p day has class in building b 
PD={}
Building=["ACC","BRI","HOH","JFF","JKP"]
for f in Prof:
    for p in P:
        for b in Building:
            PD[f,p,b]=mod.addVar(vtype=grb.GRB.BINARY,name='PD[{0},{1},{2}]'.format(f,p,b))

# add one more numeric variable Dp,f means that professor f have to show up in campus in p day of week
D={}
for f in Prof:
    for p in P:
        D[p,f]=mod.addVar(vtype=grb.GRB.BINARY,name='x[{0}]'.format(p))

#add one more varible PTi. PTi means that how many sessions of section i are in prime time
PT={}
for i in I:
    PT[i]=mod.addVar(vtype=grb.GRB.INTEGER,name='PT[{0}]'.format(i))


## Objective

In [3]:
# Objective
# As i set the constraint1 below, so the section with 2 or 3 sessions will have higher weight here
# but if i do not set constraint1, we cannot promise other sections will not take the vacancy
# for example, if DSO570 assigned to M/W 12:00, namely X[DSO570,j,"MW",12:00]=1, but X[i,j,"M",12:00]=0 and X[i,j,"W",12:00]=0, 
# so other courses maight be assign to this time too
mod.setObjective((
    # classroom utilization (max)
    sum(X[i,j,p,t]*student[i]/seat[j] for i in I for j in J for p in P for t in T)/numofsections
    # number of days that professor has class (min)
    -sum(D[p,f] for f in Prof for p in P)/numofprof
    # number of buildings that professor need to go in anyday that he has class (min
    -sum(PD[f,p,b] for f in Prof for p in P for b in Building)/numofprof
    # prime time utilization (max)
    +sum(PT[i] for i in I)/numofsections
),sense=grb.GRB.MAXIMIZE)

NameError: name 'mod' is not defined

## Constraints

In [38]:
# constraints
I_twoday=pd.Series()
I_oneday=pd.Series()
for i in I:
    if PS.get(i)==1:
        I_oneday=I_oneday.append(pd.Series(i))
    else:
        I_twoday=I_twoday.append(pd.Series(i))

In [39]:
# constraint1
# in each time slot in each day every professor can only teach one session at the same time
constraint1={}
for t in T:
    for p in P:
        for prof in Prof:
            constraint1[t,p,prof]=mod.addConstr(sum(X[i,j,p,t] for i in professor[prof].values for j in J)<=1)

In [40]:
# new constraint2
# every timeslot in every classroom in a specific day, there will be only one or zero class
constraint2={}
for j in J:
    for t in T:
        for p in P:
            constraint2[j,t,p]=mod.addConstr(sum(X[i,j,p,t] for i in I)<=1)
            
# in the next (number of required sessions)*2 timeslot all of X should be 0
I2=I
for i1 in I:
    I2=I2.drop(pd.Index(I).get_loc(i1))
    for j in J:
        for t in T:
            for p in P:
                timeslots=int(TS[i1])
                if t<=(27-timeslots): # avoid a situation that three hours course is assigned to 8:00 pm
                    #print i1,j,p,t
                    constraint2[i1,j,t,p]=mod.addConstr((X[i1,j,p,t]+sum(X[i2,j,p,t+num] for i2 in I2 for num in range(0,timeslots)))<=1,name="")

In [13]:
# this constraint3 is only for those sections whose pattern is 2, which means they have two session every week.
constraint3={}
for i in I_twoday:
    for j in J:
        for t in T:
            constraint3[i,j,"M",t]=mod.addConstr(X[i,j,"Wednesday",t]== X[i,j,"Monday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W
            
            constraint3[i,j,"T",t]=mod.addConstr(X[i,j,"Thursday",t] == X[i,j,"Tuesday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W

In [14]:
# this constraint4 is to control that when we maximum the utilization rate,the number of student registered 
#for one section should less than the classroom capacity
constraint4={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    constraint4[i,j,p,t]=mod.addConstr(X[i,j,p,t]*student[i]<=seat[j],name="")

In [15]:
# this constraint5 is to make sure that in all time in all classroom in all day, 
#for each section, sum(X)=required sessions
constraint5={}
for i in I:
    constraint5[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in T)==PS.get(i),name="")    

In [16]:
#constraint6: No Friday class for MW and TH section
constraint6={}
for i in I_twoday:
    constraint6[i] = mod.addConstr(sum(X[i,j,"Friday",t] for j in J for t in T) == 0) 

In [17]:
#constraint7:no class can last longer than the '9PM-9.30PM' block
constraint7={}
for i in I:
    constraint7[i] = mod.addConstr(sum(X[i,j,p,T.max()-ts] for j in J for p in P for ts in range(int(TS[i]))) == 0)

In [18]:
# building to classroom
ACC=[]
BRI=[]
HOH=[]
JFF=[]
JKP=[]
for j in J:
    if j[0:3]=="ACC":
        ACC.append(j)
    elif j[0:3]=="BRI":
        BRI.append(j)
    elif j[0:3]=="HOH":
        HOH.append(j)
    elif j[0:3]=="JFF":
        JFF.append(j)
    elif j[0:3]=="JKP":
        JKP.append(j)
BtoC={"ACC":ACC,"BRI":BRI,"HOH":HOH,"JFF":JFF,"JKP":JKP}

In [19]:
# constraint8 for each professor, make them teach in same building for everyday they teach
constraint8={}

for f in Prof:
    for p in P:
        for b in Building:
            courses=professor.get(f)
            classrooms=BtoC.get(b)
            for i in courses:
                for j in classrooms:
                    for t in T:
                        constraint8[f,p,b,i,j,t]=mod.addConstr(PD[f,p,b]>=X[i,j,p,t])
                        
for f in Prof:
    for p in P:
        mod.addConstr(sum(PD[f,p,b] for b in Building)<=1)

In [20]:
# constraint9 is to minimum the day that professor show in campus
constraint9={}

for f in Prof:
    for p in P:
        for i in professor.get(f):
            constraint9[f,p,i]=mod.addConstr(sum(X[i,j,p,t] for j in J for t in T)<=D[p,f])    

In [21]:
#constraint10 is to maximum the prime time of classrooms used by the course schedule
constraint10={}
for i in I:
    #print i
    constraint10[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in primetime if(t+TS[i])<=20)>=PT[i])


In [22]:
print "number of constrints "+str(len(constraint10)+len(constraint9)+len(constraint8)+len(constraint7)+len(constraint6)+len(constraint5)+len(constraint4)+len(constraint3)+len(constraint2)+len(constraint1))

number of constrints 298741


## Best solution output

In [24]:
mod.optimize()
mod.setParam('OutputFlag',False)

print('Optimal solution:',mod.ObjVal)

Optimize a model with 298781 rows, 89355 columns and 2572310 nonzeros
Variable types: 0 continuous, 89355 integer (89340 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [9e-04, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -4.3187685
Presolve removed 265506 rows and 55979 columns (presolve time = 5s) ...
Presolve removed 265518 rows and 55979 columns
Presolve time: 7.44s
Presolved: 33263 rows, 33376 columns, 360152 nonzeros
Variable types: 0 continuous, 33376 integer (33376 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 33263 rows, 33376 columns, 360152 nonzeros

Presolve removed 20 rows and 27 columns

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.5486767e+00   3.000000e+01   6.823800e+10      8s
   13525   -8.7469071e-01   0.000000e+00   0.000000e+00      9s
Concurrent 

In [25]:
# get the small subset of J that have classes in this classroom
J1=[]
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        J1.append(j)
#print J1

In [26]:
# return a blank table
def blanktable():
    T1=[]
    blank=[]
    time = datetime.timedelta(hours=8,minutes=0, seconds=0)
    for i in range(27):
        T1.append(str(time+timedelta(hours=0.5*i)))
        blank.append("")
    schedule=pd.DataFrame({"# timeslot": range(27),
                           "timeslot":T1,
                           "Monday":blank,
                           "Tuesday":blank,
                           "Wednesday":blank,
                           "Thursday":blank,
                           "Friday":blank})
    sched_cols = ['# timeslot','timeslot','Monday','Tuesday','Wednesday','Thursday','Friday']
    schedule = schedule[sched_cols]
    schedule=schedule.set_index('# timeslot')
    return schedule

In [27]:
# put the optimal solution into our beautiful schedule table
timetable={}
for j in J1:
    #print "J is "+j
    #print "schedule"+j
    schedule1=blanktable()
    for i in I:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    for a in range(int(TS[i])):
                        schedule1.loc[t+a,p]=C[i]+" / "+str(i)
                        #print schedule1
    timetable["schedule"+j]=schedule1
    #print "timetable "+"schedule"+j
    #print timetable["schedule"+j]
    

In [28]:
output=pd.ExcelWriter("schedule_24_sections.xlsx")
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        timetable["schedule"+j].to_excel(output,sheet_name=j,index=False)
output.save()

In [30]:
# make sure that every professor is assigned to one building in the day that they have class
# constraint8
for f in Prof:
    for p in P:
        for b in Building:
            if PD[f,p,b].x>0:
                print f+" "+p+" "+b+" "+str(PD[f,p,b].x)

Ryan, Tom Friday ACC 1.0
Bonner, Sarah, Elizabeth Monday BRI 1.0
Bonner, Sarah, Elizabeth Wednesday BRI 1.0
Kiddoo, Bob Tuesday ACC 1.0
Kiddoo, Bob Thursday ACC 1.0
Lennox, Clive Tuesday BRI 1.0
Lennox, Clive Thursday BRI 1.0
Owens, John, D Tuesday ACC 1.0
Owens, John, D Thursday ACC 1.0
Keller, Joe Monday BRI 1.0
Keller, Joe Wednesday BRI 1.0
Wang, Shiing-Wu Tuesday BRI 1.0
Wang, Shiing-Wu Thursday BRI 1.0
Erbstoesser, Eugene Monday BRI 1.0
Erbstoesser, Eugene Wednesday BRI 1.0


In [31]:
# check every professor's class day
# constraint9
for f in Prof:
    for p in P:
        if D[p,f].x>0:
            print f+" "+p+" "+str(D[p,f].x)

Ryan, Tom Friday 1.0
Bonner, Sarah, Elizabeth Monday 1.0
Bonner, Sarah, Elizabeth Wednesday 1.0
Kiddoo, Bob Tuesday 1.0
Kiddoo, Bob Thursday 1.0
Lennox, Clive Tuesday 1.0
Lennox, Clive Thursday 1.0
Owens, John, D Tuesday 1.0
Owens, John, D Thursday 1.0
Keller, Joe Monday 1.0
Keller, Joe Wednesday 1.0
Wang, Shiing-Wu Tuesday 1.0
Wang, Shiing-Wu Thursday 1.0
Erbstoesser, Eugene Monday 1.0
Erbstoesser, Eugene Wednesday 1.0


In [32]:
# check the number of sessions in prime time of every section
# constraint10
for i in I:
    print str(i)+" "+str(PT[i].x)

14202 1.0
14201 1.0
14206 2.0
14207 2.0
14209 2.0
14216 2.0
14224 2.0
14223 2.0
14227 2.0
14229 2.0
14228 2.0
14226 2.0
14236 2.0
14246 2.0
14260 2.0
